<a href="https://colab.research.google.com/github/HJBJalaly/DLeeg/blob/master/best%20channel/Mine/Search_1Ch_Base_Normal_FitGenerate_4Test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
!pip install mne

In [0]:
'''
Rivision:
980726:
  Differenet in/out shift Sampling
980727:
  The model is revised to addapt with fit_generate
  
  

'''

In [0]:
# Parameters:

EpochNum=20
subject_number=50
RemoveBaseLine=0
task_index = [1] # task: REO
Best1Channel=[]
TryStr='Try1'


task_number=1
task_time = 60
sampel_number_per_sec =  160 # sampel rate
total_sampel_number =  sampel_number_per_sec *task_time # 60*160
#sample_shift = 4 #step len
inner_sample_shift = 4 # inner step len
outer_sample_shift =8  # outer step len
window_len= 20
batch_size=64

Search_Space_Channel = [21,23,29,31,33,35,36,40,8,10,12,41,46,48,50,52,54,60,61,62]
Search_Space_Channel_Name=["Fp1","Fp2","F7","F3","Fz","F4","F8","T7","C3","Cz","C4","T8","P7","P3","Pz","P4","P8","O1","Oz","O2"]

# Channel Indexes:
#   Fp1,21
#   Fp2,23
#   F7,29
#   F3,31
#   Fz,33
#   F4,35
#   F8,36
#   T7,40
#   C3,8
#   Cz,10
#   C4,12
#   T8,41
#   P7,46
#   P3,48
#   Pz,50
#   P4,52
#   P8,54
#   O1,60
#   Oz,61
#   O2,62

#   T9,42
#   T10,43


In [0]:
# For Visualization
import matplotlib
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D  # noqa

# General use
import numpy as np
import math
import random

# For clear memory
import gc

#
import copy 
import csv
import pandas as pd
import datetime
import time
import os


# To download file from google drive
from google.colab import files

# For model training
from keras import layers
from keras import models
from keras import regularizers
from keras.utils import to_categorical
from keras import optimizers
from keras import backend as KerasBackend

# EEG Tools
import mne
from mne.preprocessing import create_ecg_epochs, create_eog_epochs
from mne import io
from mne import viz
from mne import Epochs, io, pick_types
from mne.event import define_target_events
from mne.time_frequency import psd_welch
print(__doc__)

In [0]:
# load dataset in array

task_number= len(task_index)

# load dataset in array
list_raw_fnames = [[0]*task_number]*subject_number
for x in range(subject_number):
  list_raw_fnames[x] = mne.datasets.eegbci.load_data(x+1,task_index,path='/content/drive/My Drive/Deep Results/Database',update_path=False)

list_rawdata = np.zeros((subject_number,task_number), dtype='object')

for i in range(subject_number):
  for j in range(task_number):
    list_rawdata[i][j] = mne.io.read_raw_edf(list_raw_fnames[i][j], preload=True)



In [0]:
# img for each subject
#subject_img_number = math.floor((total_sampel_number - sampel_number_per_sec) / sample_shift) +1 - window_len +1
subject_img_number = math.floor((total_sampel_number - sampel_number_per_sec - (window_len-1 )*inner_sample_shift) / outer_sample_shift) +1
test_start_index=subject_img_number - math.floor(subject_img_number/4) 

train_number =  test_start_index 
test_number = subject_img_number - test_start_index

subject_number_array_shuffled=np.arange(subject_number)
np.random.shuffle(subject_number_array_shuffled)


subject_img_number_array_shuffled=np.arange(subject_img_number)
np.random.shuffle(subject_img_number_array_shuffled)

print("Train sample:",train_number)
print("Test sample:",test_number)



In [0]:
def normalize_channel_data(ch , i, ch_min, ch_max):
  ch = ((ch - ch_min[i]) / (ch_max[i] - ch_min[i] ))
  return ch

def ProjectionVector(VecA,VecB):
    Projeted_VecB_on_VecA=np.dot(VecA,VecB)/np.dot(VecA,VecA)*VecA
    return Projeted_VecB_on_VecA
                  

In [0]:
#################################################################################################
## Define Train Data Generator
def GenerateTrainData(BatchSize,ListRawDataMain,SubjectNumberArrayShuffled,ListChannel,RemoveBaseLineFlag,SubjectImgNumberArrayShuffled,WindowLen,OuterSampleShift,InnerSampleShift,SampelNumberPerSec,TestStartIndex):
    
    ch_number=len(ListChannel)
    
    TrainNumber =  TestStartIndex 
    TestNumber = len(SubjectNumberArrayShuffled) - TestStartIndex

    
    Iter=0
    while True:
    
      ListRawDataTemp=ListRawDataMain.copy()
      TrainImage=np.zeros((BatchSize, ch_number, WindowLen, 160), dtype = float)
      TrainLabel=np.zeros((BatchSize),dtype=int)
      #Count=0

      # Fill Data Arrays
      for s in SubjectNumberArrayShuffled:

          DataAllChannelsRaw, times =(ListRawDataTemp[s][0][:64,:9600])


          ch_max =[]
          ch_min =[]
          # Extract Selected Channels+ Remove Baseline+ Normailze
          DataChannelsRaw   =np.zeros((ch_number,times.size),dtype = float)
          DataChannelsNormal=np.zeros((ch_number,times.size),dtype = float)
          DataChannelsOrt   =np.zeros((ch_number,times.size),dtype = float)
          for i in range(len(ListChannel)):
              DataChannelsRaw[i]=DataAllChannelsRaw[ListChannel[i]].copy()

              ch_max = np.append(ch_max, max(DataChannelsRaw[i])) # max for each cannel
              ch_min = np.append(ch_min, min(DataChannelsRaw[i])) # min for each cannel

              DataChannelsNormal[i]=normalize_channel_data(DataChannelsRaw[i].copy(), i, ch_min, ch_max)


          # Orthogonal Channel
          DataChannelsOrt[0]=DataChannelsNormal[0]
       

          for j in SubjectImgNumberArrayShuffled:
              if j <TestStartIndex :
                  for i in range(len(ListChannel)):
                      for z in range (WindowLen):
                          ExtractedData = DataChannelsOrt[i,0+j*OuterSampleShift + z*InnerSampleShift:SampelNumberPerSec+ j*OuterSampleShift + z*InnerSampleShift]

                          TrainImage[Iter][i][z] = ExtractedData# Data

                  TrainLabel[Iter] = s # label

                  Iter=Iter+1
                  #Count=Count+1
                  if Iter==BatchSize:
                    Iter=0
                    #print('Count=',Count)
                    TrainLabelClass=to_categorical(TrainLabel,len(SubjectNumberArrayShuffled))
                    yield TrainImage, TrainLabelClass 

                

In [0]:
#################################################################################################
## Define Test Data Generator
def GenerateTestData(BatchSize,ListRawDataTemp,SubjectNumberArrayShuffled,ListChannel,RemoveBaseLineFlag,SubjectImgNumberArrayShuffled,WindowLen,OuterSampleShift,InnerSampleShift,SampelNumberPerSec,TestStartIndex):
    
    ch_number=len(ListChannel)
    
    TrainNumber =  TestStartIndex 
    TestNumber = len(SubjectNumberArrayShuffled) - TestStartIndex

    TestImage=np.zeros((BatchSize, ch_number, WindowLen, 160), dtype = float)
    TestLabel=np.zeros((BatchSize),dtype=int)
    
    Iter=0
    Count=0
    
    # Fill Data Arrays
    for s in SubjectNumberArrayShuffled:

        DataAllChannelsRaw, times =(ListRawDataTemp[s][0][:64,:9600])

        ch_max =[]
        ch_min =[]
        # Extract Selected Channels+ Remove Baseline+ Normailze
        DataChannelsRaw   =np.zeros((ch_number,times.size),dtype = float)
        DataChannelsNormal=np.zeros((ch_number,times.size),dtype = float)
        DataChannelsOrt   =np.zeros((ch_number,times.size),dtype = float)
        for i in range(len(ListChannel)):
            DataChannelsRaw[i]=DataAllChannelsRaw[ListChannel[i]].copy()
            
            ch_max = np.append(ch_max, max(DataChannelsRaw[i])) # max for each cannel
            ch_min = np.append(ch_min, min(DataChannelsRaw[i])) # min for each cannel
            
            DataChannelsNormal[i]=normalize_channel_data(DataChannelsRaw[i].copy(), i, ch_min, ch_max)

    
        # Orthogonal Channel
        DataChannelsOrt[0]=DataChannelsNormal[0]
     
    
        #print('here')
        for j in SubjectImgNumberArrayShuffled:
            if j >=TestStartIndex :
                for i in range(len(ListChannel)):
                    for z in range (WindowLen):
                        #ExtractedData=DataChannelsOrt[i,0+(j+z)*sample_shift:sampel_number_per_sec+(j+z)*sample_shift]
                        ExtractedData = DataChannelsOrt[i,0+j*OuterSampleShift + z*InnerSampleShift:SampelNumberPerSec+ j*OuterSampleShift + z*InnerSampleShift]
        
                        TestImage[Iter][i][z] = ExtractedData# Data
                    
                TestLabel[Iter] = s # label
                
                Iter=Iter+1
                Count=Count+1
                if Iter==BatchSize:
                  Iter=0
                  #print('Count=',Count)
                  TestLabelClass=to_categorical(TestLabel,len(SubjectNumberArrayShuffled))
                  yield TestImage,TestLabelClass
                
                

In [0]:
#####################################################################################################
list_channel=[61]
ch_number=len(list_channel)

list_rawdataTemp=list_rawdata.copy()

#################################################################################################
# Construct Data Generator


GenTrainData=GenerateTrainData(BatchSize=batch_size,
                               ListRawDataMain=list_rawdata.copy(),
                               SubjectNumberArrayShuffled=subject_number_array_shuffled,
                               ListChannel=list_channel,
                               RemoveBaseLineFlag=RemoveBaseLine,
                               SubjectImgNumberArrayShuffled=subject_img_number_array_shuffled,
                               WindowLen=window_len,
                               OuterSampleShift=outer_sample_shift,
                               InnerSampleShift=inner_sample_shift,
                               SampelNumberPerSec=sampel_number_per_sec,
                               TestStartIndex=test_start_index)

GenTestData = GenerateTestData(BatchSize=batch_size,
                               ListRawDataTemp=list_rawdata.copy(),
                               SubjectNumberArrayShuffled=subject_number_array_shuffled,
                               ListChannel=list_channel,
                               RemoveBaseLineFlag=RemoveBaseLine,
                               SubjectImgNumberArrayShuffled=subject_img_number_array_shuffled,
                               WindowLen=window_len,
                               OuterSampleShift=outer_sample_shift,
                               InnerSampleShift=inner_sample_shift,
                               SampelNumberPerSec=sampel_number_per_sec,
                               TestStartIndex=test_start_index)

#################################################################################################
# Create Model
model = models.Sequential()
model.add(layers.Conv2D(64, (3,3), activation = 'relu', padding='same', input_shape = (ch_number,window_len,160), data_format= "channels_first" ))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(128, (3,3), activation = 'relu'))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(256, (3,3), activation = 'relu'))
model.add(layers.MaxPooling2D((2,2)))

model.add(layers.Flatten())
model.add(layers.Dense(512, activation = 'relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(subject_number, activation = 'softmax'))

model.compile(loss= 'categorical_crossentropy',
              optimizer= optimizers.RMSprop(lr= 1e-4),
              metrics = ['acc'])

#################################################################################################
# Train the Model
history = model.fit_generator(generator=GenTrainData,
                              epochs =EpochNum,
                              steps_per_epoch=np.ceil(train_number*subject_number/batch_size),
                              use_multiprocessing=False,
                              workers=1)

#################################################################################################
# Evalute the Model
TestResult=model.evaluate_generator(GenTestData,steps=np.ceil((subject_img_number-train_number-1)*subject_number/batch_size))
TestResult


In [0]:

GenTestData = GenerateTestData(BatchSize=batch_size,
                               ListRawDataTemp=list_rawdata.copy(),
                               SubjectNumberArrayShuffled=subject_number_array_shuffled,
                               ListChannel=list_channel,
                               RemoveBaseLineFlag=RemoveBaseLine,
                               SubjectImgNumberArrayShuffled=subject_img_number_array_shuffled,
                               WindowLen=window_len,
                               OuterSampleShift=outer_sample_shift,
                               InnerSampleShift=inner_sample_shift,
                               SampelNumberPerSec=sampel_number_per_sec,
                               TestStartIndex=test_start_index)
TestResult=model.evaluate_generator(GenTestData,steps=np.ceil((subject_img_number-train_number-1)*subject_number/batch_size))
TestResult

In [0]:
GenTrainDatar=   GenerateTrainData(BatchSize=2,
                                   ListRawDataMain=list_rawdata.copy(),
                                   SubjectNumberArrayShuffled=subject_number_array_shuffled,
                                   ListChannel=list_channel,
                                   RemoveBaseLineFlag=RemoveBaseLine,
                                   SubjectImgNumberArrayShuffled=subject_img_number_array_shuffled,
                                   WindowLen=window_len,
                                   OuterSampleShift=outer_sample_shift,
                                   InnerSampleShift=inner_sample_shift,
                                   SampelNumberPerSec=sampel_number_per_sec,
                                   TestStartIndex=test_start_index)
kk=0
for ii in GenTrainDatar:
  kk=kk+1
  dd,ll=ii
  #ii=np.random.rand(2,1,20,160)
  print('k=',kk)
  rr3=model.predict(x=dd)
  print('Predict index:',np.argmax(rr3[0,:]),'(value=',rr3[0,np.argmax(rr3[0,:])],'), real index=',np.argmax(ll[0,:]))
  print('Predict index:',np.argmax(rr3[1,:]),'(value=',rr3[1,np.argmax(rr3[1,:])],'), real index=',np.argmax(ll[1,:]))
  
